# Imports


Main libraries:

*   tensorflow for the Machine Learning classes
*   Numpy for maths
*   SciKit Learn hosts various datasets, we are loading the MNIST set


In [1]:
# cnn model with batch normalization for mnist
using_colab = False
import random
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization

# !! Comment these out if not using colab
from google.colab.patches import cv2_imshow
using_colab = True

# Load Dataset

In [2]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values -- this is what makes our data categorical
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# load dataset - X are the images, Y are their labels
trainX, trainY, testX, testY = load_dataset()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
if (using_colab):
  cv2_imshow(trainX[random.randint(0,100)])
  cv2_imshow(trainX[88])

# Data Prep

In [4]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

In [5]:
if (using_colab):
  cv2_imshow(trainX[random.randint(0,100)])
  cv2_imshow(trainX[88])

In [6]:
print(trainX[88].size)

784


# Define the models

This is our dense model -- the "simple" one

We begin by defining our model as a sequential process

We then flatten our data to turn it from a matrix of [28 x 28] into [784 x 1]

Our matrices must be a single dimension to be pushed through the dense layer

We then define the dense layers, we have a single input layer (Our original data), an output layer, and 3 "hidden layers".

Next, we specify our optimiser - the "learning" part

For this example, we use a categorical loss equation as we are classifying the data into 1 of 10 categories

This loss equation is how the optimiser determines how "correct" the model response was

In [7]:
# define dense nn model
def define_model1():
  model = Sequential()
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform', input_shape=(784, 1)))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

This is our convolutional model

Convolution is an image processing technique that aggregates the values of a group of nearby values, based on the kernel initialiser used

This allows for our data to maintain it's current matrix shape (so it's still a "picture") and allows the model to interpret relationships based on the location of the data.

Otherwise, this model is extremely similar to the first one - time to see how they do!

In [8]:
# define cnn model
def define_model2():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# Model Training

This is where we "train" our model and evaluate its performance (we are only checking for accuracy, but other metrics can be recorded)

This is a lot to talk about, but put simply:

We take our pictures and their corresponding labels (which number they are) and "shuffle" them like a deck of cards and deal out 5 decks of equal size

For each dataset, we then "fit" our model. We do this in "batches" of 32 images at once. Once all images are processed, we call this an "epoch"

At the completion of each epoch, the optimiser algorithm is used to adjust the **Weights and Biases** of the neurons and their connections

Our "simple model" already comprises of nearly 100,000 parameters!

In [9]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, model, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

**This will take 5+ minutes!!**

In [10]:
# evaluate model 1
model1_scores, model1_histories = evaluate_model(trainX, trainY, define_model1())

> 97.258
> 99.167
> 99.808
> 99.925
> 99.908


**This one will take even longer!!!**

In [ ]:
# evaluate model 2
model2_scores, model2_histories = evaluate_model(trainX, trainY, define_model2())

> 98.575


# Results Analysis

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		plt.xlabel('Epoch')
		# plot accuracy
		plt.subplot(2, 2, 1)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
		plt.xlabel('Epoch')
	plt.show()

In [ ]:
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: max=%.3f mean=%.3f std=%.3f, n=%d' % (max(scores)*100, mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():

	print("NN Model Results")
	# learning curves
	summarize_diagnostics(model1_histories)
	# summarize estimated performance
	summarize_performance(model1_scores)

	print("CNN Model Results")
 	# learning curves
	summarize_diagnostics(model2_histories)
	# summarize estimated performance
	summarize_performance(model2_scores)

# entry point, run the test harness
run_test_harness()